# References and Resources
* Pytorch RCNN : [ https://www.kaggle.com/julian3833/sartorius-starter-torch-mask-r-cnn-lb-0-273 ]
* TF Efficientdet : [ https://www.kaggle.com/dschettler8845/train-sartorius-segmentation-eda-effdet-tf ]
* PyTorch Detectron : P1 [ https://www.kaggle.com/slawekbiel/positive-score-with-detectron-1-3-input-data/ ], P2 [ https://www.kaggle.com/slawekbiel/positive-score-with-detectron-2-3-training/notebook ]
* Mask RCNN Paper : [  https://arxiv.org/pdf/1703.06870v3.pdf ]
* Run Length Encoding : [ https://en.wikipedia.org/wiki/Run-length_encoding#Example ]
* Open CV [ https://learnopencv.com/getting-started-with-opencv/ ]
* Pytorch UNET : [ https://www.kaggle.com/ebinan92/unet-with-deep-watershed-transform-dwt-train ]
* Keras UNET   :  [ https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-keras-u-net-training ]
* Pytorch Detectron [  ]https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training ]
* Guide to using detectron2 : [ https://www.analyticsvidhya.com/blog/2021/08/your-guide-to-object-detection-with-detectron2-in-pytorch/ ]
* captum ai segmentation tutorial : [ https://captum.ai/tutorials/Segmentation_Interpret ]
* [ https://www.tensorflow.org/tutorials/images/segmentation ] 
* TF UNET [ www.kaggle.com/arunamenon/cell-instance-segmentation-unet-eda ]
* [ https://medium.com/@ngocson2vn/a-gentle-explanation-of-backpropagation-in-convolutional-neural-network-cnn-1a70abff508b ]
* PDF - NEURONAL CELL TYPES : [https://www.cell.com/current-biology/pdf/S0960-9822(04)00440-3.pdf] 
* Youtube :UNET FOR SEGMENTATION - Digital Sreeni: [https://www.youtube.com/watch?v=lOZDTDOlqfk]
* Youtube : CELL IMAGE SEGMENTATION - MIT           : [https://www.youtube.com/watch?v=lOZDTDOlqfk]
* [https://spark-in.me/post/playing-with-dwt-and-ds-bowl-2018] 

# **Installing detectron**

* Detectron Starter [ https://www.analyticsvidhya.com/blog/2021/08/your-guide-to-object-detection-with-detectron2-in-pytorch/ ]



In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git' -q
!pip install pycocotools -q

In [ ]:
# requirements file
!pip freeze > requirements.txt

# Imports

In [ ]:
# pytorch
import torch, torchvision
print(torch.__version__)
if  torch.cuda.is_available():
    print('GPU \n')
    !nvidia-smi

In [ ]:
# general libraries
import pandas as pd
import numpy as np
import pandas as pd 
import os, json, cv2, random
from tqdm import tqdm_notebook as tqdm 
import matplotlib.pyplot as plt
# import skimage.io as io
# from pathlib import Path
import seaborn as sns


# for data preprocessing and augmentation
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


#coco 
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from joblib import Parallel, delayed
from fastcore.all import *
# detectron2
import pycocotools.mask as mask_util
from detectron2.structures import BoxMode
from detectron2 import model_zoo                 # pretrained models are available in model zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer


# data loading and utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.evaluation.evaluator import DatasetEvaluator


# filter warinings
import warnings
warnings.filterwarnings('ignore')


# Initialize the detectron2 logger and set its verbosity level to "DEBUG".
setup_logger()

**Sartorius Cell Instance Segmentation : Data Summary .**

* We have 606 Images in the Train set, and 3 Images in the public test set. Images are in PNG format.
* The annotations and paths for training images are given in the train.csv.
* The submission format is as given in sample_submission.csv.
* There are about 1900+ Images present without annotations, which can be used for semi-supervised learning at path '../input/sartorius-cell-instance-segmentation/train_semi_supervised'.
* There are also Images from LIVECell dataset with annotations, which could be included in training at path '../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021'.


In [ ]:
#directory paths 

livecell_ds = '../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021'
train_dir = '../input/sartorius-cell-instance-segmentation/train'
test_dir  =  '../input/sartorius-cell-instance-segmentation/test'


#csv files 
sample_sub = pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')
train  = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')


#checking the csv 
train.head(2)

# Simple EDA

In [ ]:
plt.figure(figsize=(10,8))

sns.countplot(data=train,x='cell_type')
plt.title('Instance of each Class in data')
plt.xlabel('Cell Type')
plt.ylabel('Cell Instance Count')
plt.show()


In [ ]:
max_an=train.groupby('id').size().sort_values(ascending=False).reset_index()
print('Maximum annotation in a image are with ID-{}: NUM(Annot) {}'.format(max_an.iloc[0]['id'],max_an[0][0]))

In [ ]:
print('Average Annotations per Image',len(train)/train['id'].nunique())

max_an=train.groupby('id').size().sort_values(ascending=False).reset_index()
print('Maximum annotation in a image are with ID-{}: NUM(Annot) {}'.format(max_an.iloc[0]['id'],max_an.iloc[0][0]))

print('Min annotation in a image are with ID-{}: NUM(Annot) {}'.format(max_an.iloc[-1]['id'],max_an.iloc[-1][0]))

In [ ]:
unique_classes = train['cell_type'].unique()

for clas in unique_classes:
    clas_df= train[train['cell_type']==clas]
    
    print('Average Annotations for class {} are {}'.format(clas,len(clas_df)/clas_df['id'].nunique()))
    
    max_an=clas_df.groupby('id').size().sort_values(ascending=False).reset_index()
    print('Max num of annot for {} ID-{}: NUM(Annot) {}'.format(clas,max_an.iloc[0]['id'],max_an.iloc[0][0]))
    print('Min num of annotation for {} with ID-{}: NUM(Annot) {}'.format(clas,max_an.iloc[-1]['id'],max_an.iloc[-1][0]))
    print('-'*20,'.'*10,'-'*20)
    

# Preparing the data in coco format.

**Getting the data ready in COCO format (Just for demonstration purpose,using a pre-prepared dataset from [ https://www.kaggle.com/slawekbiel/positive-score-with-detectron-2-3-training/notebook ].)**

In [ ]:
#from : https://www.kaggle.com/coldfir3/efficient-coco-dataset-generator?scriptVersionId=79100851

def rle2mask(rle, img_w, img_h):
    '''convert run length encoding to a image mask'''
    ## transforming the string into an array of shape (2, N)
    array = np.fromiter(rle.split(), dtype = np.uint)
    array = array.reshape((-1,2)).T
    array[0] = array[0] - 1
    
    ## decompressing the rle encoding (ie, turning [3, 1, 10, 2] into [3, 4, 10, 11, 12])
    # for faster mask construction
    starts, lenghts = array
    mask_decompressed = np.concatenate([np.arange(s, s + l, dtype = np.uint) for s, l in zip(starts, lenghts)])

    ## Building the binary mask
    msk_img = np.zeros(img_w * img_h, dtype = np.uint8)
    msk_img[mask_decompressed] = 1
    msk_img = msk_img.reshape((img_h, img_w))
    msk_img = np.asfortranarray(msk_img) ## This is important so pycocotools can handle this object
    
    return msk_img


def annotate(idx, row, cat_ids):
    
    mask = rle2mask(row['annotation'], row['width'], row['height']) # Binary mask
    c_rle = maskUtils.encode(mask) # Encoding it back to rle (coco format)
    c_rle['counts'] = c_rle['counts'].decode('utf-8') # converting from binary to utf-8
    area = maskUtils.area(c_rle).item() # calculating the area
    bbox = maskUtils.toBbox(c_rle).astype(int).tolist() # calculating the bboxes
    annotation = {
        'segmentation': c_rle,
        'bbox': bbox,
        'area': area,
        'image_id':row['id'], 
        'category_id':cat_ids[row['cell_type']], 
        'iscrowd':0, 
        'id':idx
    }
    return annotation

def coco_structure(df, workers = 6):
    
    ## Building the header
    cat_ids = {name:idx+1 for idx, name in enumerate(df.cell_type.unique())}    
    
    cats =[{'name':name, 'id':idx} for name,idx in cat_ids.items()]
    
    images = [{'id':idx, 'width':row.width, 'height':row.height, 'file_name':f'train/{idx}.png'} for idx,row in df.groupby('id').agg('first').iterrows()]
    
    # Building the annotations
    annotations = Parallel(n_jobs=workers)(delayed(annotate)(idx, row, cat_ids) for idx, row in tqdm(df.iterrows(), total = len(df)))
        
    return {'categories':cats, 'images':images, 'annotations':annotations}




#checking the code on first 1000 rows in train csv 
coco_annotations = coco_structure(train.iloc[:1000])

with open('annotations_train_sample.json', 'w', encoding='utf-8') as f:
    json.dump(coco_annotations, f, ensure_ascii=True, indent=4)

In [ ]:
# look at the generated format

print('Categories in the data',coco_annotations['categories'])
print('_'*50,'---','_'*50)
print('Images dict example', coco_annotations['images'][0])
print('_'*50,'---','_'*50)
print('Annotations dict example',coco_annotations['annotations'][0])

# Loading data 

In [ ]:
# laoding data


# path to the data preprocessed by : https://www.kaggle.com/slawekbiel
dataDir=Path('../input/sartorius-cell-instance-segmentation/')


#get configuration 
cfg = get_cfg()

# type 
cfg.INPUT.MASK_FORMAT='bitmask'

#resgister coco instance
#train
register_coco_instances('sartorius_train',{}, '../input/sartorius-cell-instance-segmentation-coco/annotations_train.json', dataDir)
#validation 
register_coco_instances('sartorius_val',{},'../input/sartorius-cell-instance-segmentation-coco/annotations_val.json', dataDir)
#all the train data 
register_coco_instances('sartorius_train_full',{},'../input/sartorius-cell-instance-segmentation-coco/annotations_all.json', dataDir)


In [ ]:
# get all train data (all 606 Images)
all_metadata = MetadataCatalog.get('sartorius_train_full')
all_dat = DatasetCatalog.get('sartorius_train_full')

# **Visualizing some samples**

In [ ]:
#select image 
ch = all_dat[5]
img = cv2.imread(ch["file_name"])

#get visual
visualizer = Visualizer(img[:, :, ::-1], metadata=all_metadata)
out = visualizer.draw_dataset_dict(ch)

#plot 
plt.figure(figsize = (20,15))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis('off')
plt.title('sample image')
plt.show()

In [ ]:
ch = all_dat[10]
img = cv2.imread(ch["file_name"])


visualizer = Visualizer(img[:, :, ::-1], metadata=all_metadata)
out = visualizer.draw_dataset_dict(ch)

plt.figure(figsize = (20,15))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis('off')
plt.title('sample image')
plt.show()

In [ ]:
ch = all_dat[100]
img = cv2.imread(ch["file_name"])


visualizer = Visualizer(img[:, :, ::-1], metadata=all_metadata)
out = visualizer.draw_dataset_dict(ch)

plt.figure(figsize = (20,15))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis('off')
plt.title('sample image')
plt.show()

# Metrics and Training Helper functions

In [ ]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou


def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)
    

# Setting Config for the model

In [ ]:
#getting Mask RCNN architecture -get config file for RCCN_resnet50
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

#set training and test data dir

#using all the data for training
cfg.DATASETS.TRAIN = ("sartorius_train_full",)

#validation data(inc in training)
cfg.DATASETS.TEST = ("sartorius_val",)


#num workers
cfg.DATALOADER.NUM_WORKERS = 2

# loading pretrained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

#default
cfg.SOLVER.IMS_PER_BATCH = 2

#learning rate for base model
cfg.SOLVER.BASE_LR = 0.001

# number of iterations
cfg.SOLVER.MAX_ITER = 7500

#Region of Interest batch size
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512

#number of output classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

#threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.SOLVER.WARMUP_ITERS = 10 #How many iterations to go from 0 to reach base LR
cfg.SOLVER.STEPS = (2000,4000,6000) #change the LR 0.25,0.5
cfg.TEST.EVAL_PERIOD = 250
#save model
cfg.SOLVER.CHECKPOINT_PERIOD=2000

cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train_full')) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

In [ ]:
#training

#make a directory to store model outputs
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

#intanciate trainer with configs from pervious cells
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=True)

#start train
trainer.train()

**Plotting History**

In [ ]:
metrics_df = pd.read_json("./output/metrics.json",
                          orient="records",
                          lines=True).sort_values("iteration")

#get values 
x1 = metrics_df.iteration
l_cls=metrics_df.loss_cls
l_msk=metrics_df.loss_mask
l_bb=metrics_df.loss_box_reg


plt.subplots(figsize=(16,8))
plt.plot(x1,l_cls,color='b',label='Class_loss')
plt.plot(x1,l_msk,color='k',label='Mask_loss')
plt.plot(x1,l_bb,color='g',label='BBox_loss')

plt.title('Train Summary')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Loss')

plt.show()

# PLotting predicted Images against Original Images

In [ ]:
#set dpi 
plt.rcParams['figure.dpi'] = 300

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # set a custom testing threshold


#predict (with the model we trained above)
predictor = DefaultPredictor(cfg)

#get tvalidataion data for eval
dataset_dicts = DatasetCatalog.get('sartorius_val')



outp,outt = [],[]  # predicted, original

#selecting 6 sampeles to evaluate on 
for d in random.sample(dataset_dicts, 6):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    
    #visualize
    v = Visualizer(img[:, :, ::-1],
                   metadata = MetadataCatalog.get('sartorius_train'), 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    
    
    out_pred = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    visualizer = Visualizer(img[:, :, ::-1])
    out_target = visualizer.draw_dataset_dict(d)
    outp.append(out_pred)
    outt.append(out_target)

    plt.subplots(6,2,figsize=(30,60))

    
n=1
for i in range(len(outp)):
    plt.subplot(6,2,n)
    plt.imshow(outp[i].get_image())
    plt.axis('off')
    plt.title('Predicted(Thresh=0.5)')
    n+=1
    
    plt.subplot(6,2,n)
    plt.imshow(outt[i].get_image())
    plt.axis('off')
    plt.title('Original')
    if n==12:
        break
    n+=1
    
    
plt.tight_layout()
plt.show()
    

**Visualizing with a different prediction threshold**

**Thresh = 0.35**

In [ ]:
# changing pred thresh
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.35


#predictor
predictor = DefaultPredictor(cfg)


outp,outt = [],[]  # predicted, original

#selecting 5 sampeles to evaluate on 
for d in random.sample(dataset_dicts, 10):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    
    #visualize
    v = Visualizer(img[:, :, ::-1],
                   metadata = MetadataCatalog.get('sartorius_train'), 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    
    
    out_pred = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    visualizer = Visualizer(img[:, :, ::-1])
    out_target = visualizer.draw_dataset_dict(d)
    outp.append(out_pred)
    outt.append(out_target)


    
    
plt.subplots(6,2,figsize=(30,60))
#plot
n=1
for i in range(len(outp)):
    plt.subplot(6,2,n)
    plt.imshow(outp[i].get_image())
    plt.axis('off')
    plt.title('Predicted(Thresh=0.5)')
    n+=1
    
    plt.subplot(6,2,n)
    plt.imshow(outt[i].get_image())
    plt.axis('off')
    plt.title('Original')
    if n==12:
        break
    n+=1
    
    
plt.tight_layout()
plt.show()
    

**Thresh = 0.25**

In [ ]:
# changing pred thresh
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25


#predictor
predictor = DefaultPredictor(cfg)


outp,outt = [],[]  # predicted, original

#selecting 5 sampeles to evaluate on 
for d in random.sample(dataset_dicts, 10):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    
    #visualize
    v = Visualizer(img[:, :, ::-1],
                   metadata = MetadataCatalog.get('sartorius_train'), 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    
    
    out_pred = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    visualizer = Visualizer(img[:, :, ::-1])
    out_target = visualizer.draw_dataset_dict(d)
    outp.append(out_pred)
    outt.append(out_target)
    

    #plot
plt.subplots(6,2,figsize=(30,60))

n=1
for i in range(len(outp)):
    plt.subplot(6,2,n)
    plt.imshow(outp[i].get_image())
    plt.axis('off')
    plt.title('Predicted(Thresh=0.25)')
    n+=1
    
    plt.subplot(6,2,n)
    plt.imshow(outt[i].get_image())
    plt.axis('off')
    plt.title('Original')
    if n==12:
        break
    n+=1
    
    
plt.tight_layout()
plt.show()
    

**Thresh = 0.15**

In [ ]:
# changing pred thresh
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15


#predictor
predictor = DefaultPredictor(cfg)


outp,outt = [],[]  # predicted, original

#selecting 5 sampeles to evaluate on 
for d in random.sample(dataset_dicts, 10):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    
    #visualize
    v = Visualizer(img[:, :, ::-1],
                   metadata = MetadataCatalog.get('sartorius_train'), 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    
    
    out_pred = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    visualizer = Visualizer(img[:, :, ::-1])
    out_target = visualizer.draw_dataset_dict(d)
    outp.append(out_pred)
    outt.append(out_target)
    
    
#plot
plt.subplots(6,2,figsize=(30,60))
n=1
for i in range(len(outp)):
    plt.subplot(6,2,n)
    plt.imshow(outp[i].get_image())
    plt.axis('off')
    plt.title('Predicted(Thresh=0.15)')
    n+=1
    
    plt.subplot(6,2,n)
    plt.imshow(outt[i].get_image())
    plt.axis('off')
    plt.title('Original')
    if n==12:
        break
    n+=1
    
    
plt.tight_layout()
plt.show()

# Sample Inference

In [ ]:

# path to competiton directory
comp_dir = Path('../input/sartorius-cell-instance-segmentation')

# get test names 
test_paths = (comp_dir/'test').ls()

print('Number of Images in Test', len(test_paths))
print('Test Images Paths  \n',test_paths)

**Helper functions**

In [ ]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


In [ ]:
def load_img(path):
    im = cv2.imread(str(path))
    return im
    


def predict_masks(path,
                  Predictor_Model, # predicted model
                  class_thresh, # threshold for each class
                  pixel_thresh): # pixel threshold for each class
    
    im = load_img(path)
    pred = Predictor_Model(im)#get prediction
    pred_class = torch.mode(pred['instances'].pred_classes)[0] # get classes
    
    take = pred['instances'].scores >= class_thresh[pred_class] # segment pixels based on what class it is from
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    used = np.zeros(im.shape[:2], dtype=int) 
    
    res = []
    
    for mask in pred_masks:
        mask = mask * (1-used)
        if mask.sum() >= pixel_thresh[pred_class]: # skip predictions with small area
            used += mask
            res.append(rle_encode(mask))
            
    return res

**Set thresholds and configure predictor**

In [ ]:
# set params 

detect_per_img = 600


# using the thresholds as author..will try to tweak them later 
clss=['shsy5y','astro','cort']

class_thresh= [.15, .35, .55]
thresholds_dict = dict(zip(clss,class_thresh))
#pixel_thresh
pixel_thresh= [75, 150, 75]

# for predictions
ids, masks=[],[]  # img ids, mask annotations

img_masks = []

# path to best predictor model
predictor_model_path = './output/model_final.pth'

In [ ]:
#config dict
cfg = get_cfg()
#get model arch
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#mask format
cfg.INPUT.MASK_FORMAT='bitmask'
#num of classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
# weights from training notebook model
cfg.MODEL.WEIGHTS = predictor_model_path
#num of detections per image
cfg.TEST.DETECTIONS_PER_IMAGE = detect_per_img


**Inference**

In [ ]:
predictor = DefaultPredictor(cfg)

for path in test_paths:
    encoded_masks = predict_masks(path=path,
                                  Predictor_Model=predictor, # predicted model
                                  class_thresh = class_thresh, # threshold for each class
                                  pixel_thresh = pixel_thresh)      # pixel threshold(area) for each class

    img_masks.append(encoded_masks) # append masks for further visualization
    for encoding in encoded_masks:
        #save img ids
        ids.append(path.stem)
        # save individual encodings
        masks.append(encoding)

        
        
print('Number of Annotations predicted for 3 Test Images are ',len(masks))

In [ ]:
def plot_predcted_mask(img_path,
                       encoded_mask,
                       thresholds_dict=thresholds_dict):
    '''plot mask with imag'''
    print('Thresholds used are ',thresholds_dict)
    
    
    fig,axs=plt.subplots(1,2,figsize=(16,8))
    
    img = load_img(img_path)
    
    axs[0].imshow(img)
    plt.axis('off')
    axs[0].set_title('Test Image')
    
    for enc in encoded_masks:
        decoded = rle_decode(enc)
        axs[1].imshow(np.ma.masked_where(decoded==0, decoded))
        plt.axis('off')
        axs[1].set_title('Predicted Mask')
    
    plt.tight_layout()
    plt.show()
    

# Plotting predicted Images


In [ ]:
print('Test Image 1')
plot_predcted_mask(img_path =test_paths[0],encoded_mask=img_masks[0])

In [ ]:
print('Test Image 2')
plot_predcted_mask(img_path =test_paths[1],img_masks[1])

In [ ]:
print('Test Image 3')
plot_predcted_mask(img_path =test_paths[2],img_masks[2])

**save some models(and delete the rest)**

In [ ]:
#saving these output files
save_list = ['./output/events.out.tfevents.1639574743.943df9329b01.34.0',
             './output/metrics.json',
             './output/model_0002999.pth',
            './output/model_final.pth']

#path where outputs of detectron2 are saved
source = './output'
#path to move 
destination='./detectron_outputs'
os.mkdir(destination)


for file_path in save_list:
    shutil.copy(file_path,destination)
    

In [ ]:
#delete other saved files 
shutils.rmtree(source)